# Utilities

## Imports

In [2]:
import os
import fnmatch
import sys
import csv
import math
import h5py

operating_system = sys.platform

## Locations

In [ ]:
# This file is for turning the h5 data to a compatible csv file.
# In writeToCSV the data (in dictionaries) is written in a csv
# In convertH5Data the data from h5 is handled to calculate the coordinates and putting the data in a dictionary

# Change the code below to the folder/path of the h5 files (location)
# Target_location is where the csv data are written to
location = 'RadarData'
target_location = 'csvData'

## Add heights

Add heights voegt de data van de radarbeelden van verschillende elevation angles samen tot een lijst met dictionaries met DBZH waarden en coordinaten.

In [ ]:
def add_heights(datasets):
    points = []
    for data in datasets:
        if (data != "how" and data != "what" and data != "where"):
            elevation_angle = datasets[data]["where"].attrs.values()[1]
            nbins = datasets[data]["where"].attrs.values()[2]
            nrays = datasets[data]["where"].attrs.values()[3]
            bin_distance = datasets[data]["where"].attrs.values()[4]
            data_DBZH = datasets[data]["data1"]["data"][()]
            data_TH = datasets[data]["data2"]["data"][()]
            data_VRAD = datasets[data]["data3"]["data"][()]
            offsets = [datasets[data]["data1"]["what"].attrs.values()[1], datasets[data]["data2"]["what"].attrs.values()[1], datasets[data]["data3"]["what"].attrs.values()[1]]
            gains = [datasets[data]["data1"]["what"].attrs.values()[0], datasets[data]["data2"]["what"].attrs.values()[0], datasets[data]["data3"]["what"].attrs.values()[0]]
            points.extend(get_coordinates_picture(nbins, nrays, elevation_angle, bin_distance, data_DBZH, data_TH, data_VRAD, offsets, gains))
    return points

## Get coordinates picture
Get coordinates picture brekent van elke ray de XYZ coordinaten ens laat deze in een lijst van dictionaries op samen met de DBZH waardes.

In [ ]:
# get_coordinates_picture berekent van elk punt in een radarafbeelding de bijbehorende x,y en z coordinaat
# deze worden opgeslagen in een lijst van dictionaries samen met de DBZH waarde van elk punt.
def get_coordinates_picture(nbins, nrays, elevation_angle, bin_distance, data_DBZH, data_TH, data_VRAD, offsets, gains):
    pic = []
    for bin_number in range(nbins):
        for ray_number in range(nrays):
            xyz = get_xyz(elevation_angle, bin_number, bin_distance, ray_number, nrays)
            tmp = {"DBZH": offsets[0] + data_DBZH[ray_number][bin_number]*gains[0], "TH":offsets[1] + data_TH[ray_number][bin_number]*gains[1],
            "VRAD":offsets[2] + data_VRAD[ray_number][bin_number] * gains[2], "x": xyz[0], "y": xyz[1], "z": xyz[2]}
            pic.append(tmp)
    return pic

## Get XYZ
get_xyz berekent de x, y en z coordinaat van een radarbin. 
  
INPUT:  
elevation angle: de hoek van de radarbeam t.o.v. de grond  
bin_number: de n-de radarbin van een beam  
bin_distance: de afstand tussen de radarbins  
angle_number: de n-de beam van de radar in de x-y richting  
total_angles: het totaal aantal beams wat is genomen in een rondje  
  
OUTPUT:  
de x, y, en z coordinaat van de radabin

In [ ]:
def get_xyz(elevation_angle, bin_number, bin_distance, angle_number, total_angles):
    z = bin_number * bin_distance * math.sin(math.radians(elevation_angle))
    dis = bin_number * bin_distance * math.cos(math.radians(elevation_angle))
    beam_angle = (math.radians(float(angle_number)/total_angles) * 360)
    y = dis * math.cos(beam_angle)
    x = dis * math.sin(beam_angle)
    return [x,y,z]

## Convert H5 file to CSV

In [3]:
def convertH5toCSV(h5file, csvfileName):
    # Get data from h5 file
    datasets = h5py.File(h5file, 'r')

    # Convert h5 file to xyz
    points = add_heights(datasets) # From convertH5Data

    # Export points to csv file
    createCSVfile(csvfileName, points) # From writeToCSV

## Import Radar Data to CSV

In [ ]:
def importData(location, newpath):
    matches = []
    # Get all h5 radar files in the radarData directory
    for root, dirnames, filenames in os.walk(location):
        for filename in fnmatch.filter(filenames, '*.h5'):
            matches.append(os.path.join(root, filename))
    if operating_system == 'windows':
        # Convert all H5 files to csv files
        for h5file in matches:
            split = h5file.split('\\') # [day, hour, quarter of an hour]
            if not os.path.exists(newpath+split[1]):
                os.makedirs(newpath+split[1])
            csvFileName = newpath+"\\"+split[1]+"\\"+split[1]+"-"+split[2]+"-"+split[3]+".csv"
            convertH5toCSV(h5file, csvFileName)
    else:
        # Convert all H5 files to csv files
        for h5file in matches:
            split = h5file.split('/') # [day, hour, quarter of an hour]
            regexResult = split
            if not os.path.exists(newpath+"/"+split[1]):
                os.makedirs(newpath+"/"+split[1])
            csvFileName = newpath+"/"+split[1]+"/"+split[1]+"-"+split[2]+"-"+split[3]+".csv"
            convertH5toCSV(h5file, csvFileName)


def main(argv):
    print "******************** Creating CSV files *******************************"
    importData(location, target_location);
    print "********************* CSV files created ********************************"
    pass

if __name__ == "__main__":
    main(sys.argv)